In [ ]:
from config import dogs_vs_cats_config as config
from pyimage.preprocessing.preprocessors import ImageToArrayPreprocessor, SimplePreprocessor, MeanPreprocessor, CropPreprocessor
from pyimage.io.hd5py import HDF5DatasetGenerator
from tensorflow.keras.models import load_model
import numpy as np
import progressbar
import json

In [ ]:
means = json.loads(open(config.DATASET_MEAN).read())

sp = SimplePreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], mean["G"], means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

print("INFO loading model")
model = load_model(config.MODEL_PATH)

In [ ]:
print("INFO predicting on test data (no crops)")
testGen = HDF5Generator(config.TEST.HDF5, 64, preprocessors=[sp, mp, iap], classes=2)
predictions = model.predict_generator(testGen.generator(),
                                     steps=testGen.numImages//64, max_queue_size=10)

(rank1, _) = rank5_accuracy(predictions, testGen.db["labels"])
print("INFO rank-1: {:.2f}%".format(rank1*100))
testGen.close()

In [ ]:
testGen = HDF5DatasetGenerator(config.TEST_HDF5, 64, preprocessors=[mp], classes=2)
predictions = []

widgets = ["Evaluating: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testGen.numImages // 64, widgets=widgets).start()

In [ ]:
for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
    # loop over each of the individual images
    for image in images:
        # apply the crop preprocessor to the image to generate 10
        # separate crops, then convert them from images to arrays
        crops = cp.preprocess(image)
        crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")
        
        pred = model.predict(crops)
        predictions.append(pred.mean(axis=0))
    
    pbar.update(i)

In [ ]:
pbar.finish()
print("[INFO] predicting on test data (with crops)...")
(rank1, _) = rank5_accuracy(predictions, testGen.db["labels"])
print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
testGen.close()